# Experience 4

**Name**:  Tina Yung-Fang Tu
**Time Spent**: 15 hr

In [1]:
import numpy as np
import pandas as pd
import itertools as it
from scipy.stats import chi2

In [2]:
# please set the path to your data directory here
path = "./EPI511/"

# please use the following function (or something like it) to read files
def pname(name):
    '''Prepend the path to the filename'''
    return path + '/' + name

def popen(name):
    '''Open file in the path'''
    return open(pname(name))

In [3]:
#################### functions to read in data ##################
def read_snp(file):
    '''Read a snp file into a pandas dataframe'''
    return(pd.read_table(
        file,
        sep='\s+', # columns are separated by whitespace
        # names of the columns
        names=[None, 'chromosome', 'morgans', 'position', 'ref', 'alt'],
        index_col=0))

SNPs = read_snp(path + 'HapMap3.snp') 

def get_chr_range(chromosome):
    '''Returns the range of positions where SNPs for a chromosome are kept'''
    filt = SNPs.query('chromosome=={}'.format(chromosome))
    start = SNPs.index.get_loc(filt.iloc[0].name)
    stop  = SNPs.index.get_loc(filt.iloc[-1].name) + 1
    return(start, stop)

def read_geno(file):
    '''Reads a geno file into a masked numpy matrix'''
    return(np.genfromtxt(
        file,               # the file
        dtype='uint8',      # read the data in as 1-byte integers
        delimiter=1,        # 1-byte width data
        missing_values=9,   # 9 indicates missing data
        usemask=True        # return a masked array
    ))

def read_geno_pop_chr(pop, chromosome):
    '''Reads a slice of a geno file into a masked numpy matrix'''
    f = open(path + pop + '.geno')      # open the file
    (start, stop) = get_chr_range(chromosome)
    s = it.islice(f, start, stop) # slice the file only keeping SNPs of chr
    return read_geno(s)

## (1) Scan for selection

Conduct a genome-wide scan for selection based on unusual population differentiation for (a) CEU vs. TSI (assume genome-wide FST = 0.004), (b) CHB vs. JPT (assume FST = 0.007), and (c) CEU vs. CHB (assume FST = 0.11).  In each case, print output only for suggestive SNPs attaining a χ2(1 dof) statistic > 20, with at most one most significant SNP per chromosome.  Print the allele frequencies in each population as well as the χ2(1 dof) statistic, and indicate which signals are genome-wide significant (P-value < 5 x 10-8).  Discuss results of (a) vs. (b) vs. (c). 
Note: here, “genome-wide” refers to the autosomes (chromosomes 1-22).



### Solution

In [4]:
def calculate_sigma_d_squared(pop1, pop2, chr, Fst):
    pop1_geno = read_geno_pop_chr(pop1, chr)
    pop2_geno = read_geno_pop_chr(pop2, chr)

    # Compute allele frequencies excluding masked values
    pop1_af = pop1_geno.mean(axis=1).filled(-1) / 2
    pop2_af = pop2_geno.mean(axis=1).filled(-1) / 2
    
    # Calculate N for each SNP, excluding masked values
    N1 = (~pop1_geno.mask).sum(axis=1)
    N2 = (~pop2_geno.mask).sum(axis=1)
    
    # Select valid SNPs present in both populations
    valid_snps = (pop1_af >= 0) & (pop2_af >= 0)
    pop1_af, pop2_af = pop1_af[valid_snps], pop2_af[valid_snps]
    N1, N2 = N1[valid_snps], N2[valid_snps]
    
    # Compute average allele frequency
    p = (pop1_af + pop2_af) / 2
    d = (2 * Fst + 1/(2*N1) + 1/(2*N2)) * p * (1 - p)
    
    # Compute chi-square statistic
    chi2_statistic = (pop1_af - pop2_af) ** 2 / d
    p_values = chi2.sf(chi2_statistic, df=1)  
    
    # Identify significant SNPs
    suggestive_threshold = 20 
    significant_snps = np.where(chi2_statistic > suggestive_threshold)[0]

    # If there are suggestive SNPs, print the most significant one
    if len(significant_snps) > 0:
        max_index = significant_snps[np.argmax(chi2_statistic[significant_snps])]  

        print(f"Chr {chr}, SNP {max_index}, pop1_af: {pop1_af[max_index]:.4f}, pop2_af: {pop2_af[max_index]:.4f}, χ²: {chi2_statistic[max_index]:.2f}, p-value: {p_values[max_index]:.2e}")
        
        # Check genome-wide significance
        genome_wide_threshold = chi2.isf(5e-8, df=1)
        if chi2_statistic[max_index] > genome_wide_threshold:
            print('Genome-wide significant')

In [5]:
# Compute for chromosomes 1-22
print('CEU vs. TSI')
for chr in range(1, 23):
    calculate_sigma_d_squared('CEU', 'TSI', chr, 0.004)
print('----------------------------------------------')

print('CHB vs. JPT')
for chr in range(1, 23):
    calculate_sigma_d_squared('CHB', 'JPT', chr, 0.007)
print('----------------------------------------------')

print('CEU vs. CHB')
for chr in range(1, 23):
    calculate_sigma_d_squared('CEU', 'CHB', chr, 0.11)

CEU vs. TSI
Chr 2, SNP 33858, pop1_af: 0.2054, pop2_af: 0.7557, χ²: 66.86, p-value: 2.91e-16
Genome-wide significant
Chr 6, SNP 10949, pop1_af: 0.3604, pop2_af: 0.6648, χ²: 20.39, p-value: 6.30e-06
Chr 14, SNP 10947, pop1_af: 0.8348, pop2_af: 0.5511, χ²: 20.84, p-value: 4.98e-06
Chr 15, SNP 1464, pop1_af: 0.1216, pop2_af: 0.4034, χ²: 22.55, p-value: 2.04e-06
----------------------------------------------
CHB vs. JPT
Chr 6, SNP 9826, pop1_af: 0.5893, pop2_af: 0.9186, χ²: 22.69, p-value: 1.90e-06
Chr 8, SNP 37819, pop1_af: 0.7500, pop2_af: 0.3779, χ²: 21.85, p-value: 2.95e-06
----------------------------------------------
CEU vs. CHB


*comments*

From the results, we can observe that we detect more SNPs in CEU vs. TSI, as this group has the smallest Fst value (0.004). This is consisitent with the fact that population differentiation test for selection is more powerful in closely related populations (small Fst). Also, when the allele frequency difference between population is large, the χ² statistic is more likely to reach significance. On the other hand, population differentiation test for selection has limited statistical power when used on groups with larger Fst. 

## (2)  LD scan

How far does LD (r2>0.5) with LCT SNP rs13404551 on chr 2 span in the CEU population? (In each chromosomal direction, considered separately, how far is the farthest SNP in LD?) Repeat these computations for the populations TSI, CHB, YRI.  Why does LD vary across these populations?  In the CEU population, for each of the two alleles of the LCT SNP rs13404551, restrict to CEU individuals who are homozygous for that allele (2 copies of that allele) and then compute the homozygosity (proportion of SNPs with 0 or 2 copies of reference allele) within that set of individuals in the region 1Mb to the left of rs13404551 and in the region 1Mb to the right of rs13404551. Discuss results for the two alleles.

### Solution

In [6]:
def get_snp_info(pop, snp, chr):
    pop_SNPs = read_snp(path + f'{pop}.snp')  
    pop_chr_snps = pop_SNPs[pop_SNPs['chromosome'] == chr]
    if snp in pop_chr_snps.index: 
        snp_index = pop_chr_snps.index.get_loc(snp)
        snp_position = pop_chr_snps.loc[snp, 'position']
    return snp_index, snp_position


def compute_r2(snp1, snp2):
    # Use masks to filter valid data
    valid = ~snp1.mask & ~snp2.mask
    if np.sum(valid) < 2:
        return np.nan

    gA = snp1[valid].astype(float)
    gB = snp2[valid].astype(float)
    
    if np.std(gA) == 0 or np.std(gB) == 0:
        return np.nan
    
    # Genotype count for allele
    gA_mean = np.mean(gA) 
    gB_mean = np.mean(gB) 

    Na = len(gA)
    Nb = len(gB)

    var_a = ((gA - gA_mean)**2).sum()/Na
    var_b = ((gB - gB_mean)**2).sum()/Nb

    # Compute r^2
    numerator = ((gA *gB).mean() - (gA_mean * gB_mean))**2
    denominator = var_a * var_b
    if denominator == 0:
        return np.nan
    return numerator / denominator

In [7]:
def compute_ld_span(pop, snp, chr):
    geno_data = read_geno_pop_chr(pop, chr)
    snp_index, snp_position = get_snp_info(pop, snp, chr)

    pop_SNPs = read_snp(path + f'{pop}.snp')
    pop_chr_snps = pop_SNPs[pop_SNPs['chromosome'] == chr]
    snp_positions = pop_chr_snps['position'].values

    # Compute LD (r²) for all SNPs
    ld_values = [compute_r2(geno_data[snp_index], geno_data[i]) for i in range(geno_data.shape[0])]

    # Identify SNPs with r² > 0.5
    high_ld_snps = [i for i, r2 in enumerate(ld_values) if r2 > 0.5]

    # Determine the farthest SNPs in each direction
    leftmost_snp = min((i for i in high_ld_snps if i < snp_index), default=snp_index)
    rightmost_snp = max((i for i in high_ld_snps if i > snp_index), default=snp_index)

    # Compute distances
    left_distance = abs(snp_positions[leftmost_snp] - snp_position)
    right_distance = abs(snp_positions[rightmost_snp] - snp_position)

    print(f'Population: {pop}')
    print(f"LD (r² > 0.5) with SNP {snp} spans {left_distance} bp upstream and {right_distance} bp downstream.")

In [8]:
# Run for rs13404551 in CEU population on chromosome 2
compute_ld_span("CEU", "rs13404551", 2)
compute_ld_span("TSI", "rs13404551", 2)
compute_ld_span("CHB", "rs13404551", 2)
compute_ld_span("YRI", "rs13404551", 2)

Population: CEU
LD (r² > 0.5) with SNP rs13404551 spans 959566 bp upstream and 95621 bp downstream.
Population: TSI
LD (r² > 0.5) with SNP rs13404551 spans 164676 bp upstream and 64656 bp downstream.
Population: CHB
LD (r² > 0.5) with SNP rs13404551 spans 199285 bp upstream and 33689 bp downstream.
Population: YRI
LD (r² > 0.5) with SNP rs13404551 spans 30170 bp upstream and 33689 bp downstream.


In [9]:
# Compute the homozygosity in the region ±1Mb of SNP rs13404551 separately for individuals homozygous for each allele.
def compute_homozygosity_for_snp(pop, snp, chr):
    geno_data = read_geno_pop_chr(pop, chr)
    snp_index, snp_position = get_snp_info(pop, snp, 2)
 
    # Identify individuals that are homozygous for each allele at rs13404551
    ref_homozygotes = np.where(geno_data[snp_index, :] == 2)[0]
    alt_homozygotes = np.where(geno_data[snp_index, :] == 0)[0]
    
    # Load SNP position information for chromosome 2.
    pop_SNPs = read_snp(path + f'{pop}.snp')
    pop_chr_snps = pop_SNPs[pop_SNPs['chromosome'] == chr]
    snp_positions = pop_chr_snps['position'].values  
    
    # Indices for SNPs to the left and right of rs13404551 within 1Mb
    left_region_indices = np.where(snp_positions < snp_position)[0]
    right_region_indices = np.where(snp_positions > snp_position)[0]
    
    left_region_indices = left_region_indices[(snp_positions[left_region_indices] >= (snp_position - 1_000_000))]
    right_region_indices = right_region_indices[(snp_positions[right_region_indices] <= (snp_position + 1_000_000))]
    
    # Subset the genotype data
    geno_left_region = geno_data[left_region_indices, :]
    geno_right_region = geno_data[right_region_indices, :]
    
    geno_left_ref = geno_left_region[:, ref_homozygotes]
    geno_left_alt = geno_left_region[:, alt_homozygotes]
    geno_right_ref = geno_right_region[:, ref_homozygotes]
    geno_right_alt = geno_right_region[:, alt_homozygotes]
    
    # Helper function 
    def compute_group_homozygosity(geno_region_group):
        valid_mask = ~geno_region_group.mask
        valid_genotypes = geno_region_group[valid_mask]
        homozygous_calls = np.isin(valid_genotypes, [0, 2])
        return np.sum(homozygous_calls) / homozygous_calls.size
    
    # Compute the homozygosity for each group and region
    homozygosity_left_ref = compute_group_homozygosity(geno_left_ref)
    homozygosity_left_alt = compute_group_homozygosity(geno_left_alt)
    homozygosity_right_ref = compute_group_homozygosity(geno_right_ref)
    homozygosity_right_alt = compute_group_homozygosity(geno_right_alt)

    return homozygosity_left_ref, homozygosity_left_alt, homozygosity_right_ref, homozygosity_right_alt


homo_left_ref, homo_left_alt, homo_right_ref, homo_right_alt = compute_homozygosity_for_snp('CEU', 'rs13404551', 2)
print(f"Homozygosity to the left for genotype 2: {homo_left_ref}")
print(f"Homozygosity to the left for genotype 0: {homo_left_alt}")
print(f"Homozygosity to the right for genotype 2: {homo_right_ref}")
print(f"Homozygosity to the right for genotype 0: {homo_right_alt}")

Homozygosity to the left for genotype 2: 0.555375909458367
Homozygosity to the left for genotype 0: 0.9483495145631068
Homozygosity to the right for genotype 2: 0.5879036391594055
Homozygosity to the right for genotype 0: 0.7196220286431418


*comments*

From the LD span results, we see that European populations exhibit longer LD blocks due to historical bottlenecks and less genetic recombination compared to African populations. The long LD span indicated longer haplotypes. YRI has shorter LD due to greater genetic diversity and its ancestral role. TSI and CHB have intermediate length of LD. The homozygosity to the left of rs13404551 for genotype 0 is very high, showing that individuals in this population share a long haplotype in the region. Relatively lower homozygosity to the right of rs13404551 for genotype 0 shows that there are more haplotype diversity. Homozygosity for genotype 2 in both directions are relatively lower as compared to genotype 0, meaning that there are more haplotype diveristy and more recombination events in the allele. 

## (3)  

Consider a set of 1,000 unlinked SNPs in the 113 YRI individuals defined by the first SNP and every 50th SNP thereafter (SNPs 0, 50, …, 49950, if the first SNP is SNP 0).  Assign the first 100 SNPs as causal SNPs and other 900 SNPs as non-causal SNPs. Simulate quantitative phenotypes for YRI individuals by assuming that causal SNPs have effect size per normalized genotype = 0.1 (note that this is different from effect size per allele) and null SNPs have effect size per normalized genotype = 0, for a total hg2 of 100 x (0.1)2 = 1.00.  (If hg2 were less than 1, the simulated phenotypes would need to include a term with variance 1–hg2 for the variance not explained by genotyped SNPs, however, that is not the case here.)  Compute ATT χ2 association statistics for each of the 1,000 SNPs.  What is the average χ2 for causal SNPs, what is the average χ2 for null SNPs, what is the average χ2 for all SNPs?  Do these agree with the derivations provided in Week 8 slides? (Note: when computing normalizing genotypes, missing data should be set to 0 after normalization.)

### Solution

In [10]:
def armitage_trend_test(genotype, phenotype):
    # Remove missing data
    valid = ~genotype.mask 
    valid_genotype = genotype[valid]
    valid_phenotype = phenotype[valid] 

    if len(valid_genotype) == 0:
        return np.nan
    # Compute correlation coefficient
    r = np.corrcoef(valid_genotype, valid_phenotype)[0, 1]
    if np.isnan(r):
        return np.nan
    
    # Compute chi-square statistic
    N = len(valid_genotype)
    return N * r**2

In [11]:
YRI_geno_chr1 = read_geno_pop_chr('YRI', 1)

# Define causal SNPs
snp_indices = np.arange(0, 50000, 50)
YRI_geno_subset = YRI_geno_chr1[snp_indices, :]

causal_labels = np.zeros(1000, dtype=bool)
causal_labels[:100] = True

# Initialize an array to hold the phenotype for each individual.
n = YRI_geno_subset.shape[1]
Y_YRI = np.zeros(n)
effect_size = 0.1

# Create a 2D matrix to store normalized genotypes
X_YRI = np.zeros_like(YRI_geno_subset, dtype=float)

# Standardize all SNPs and compute phenotype contribution
for i in range(1000):  
    snp_genotypes = YRI_geno_subset[i, :]
    
    # Compute allele frequency and standard deviation
    mean_val = np.ma.mean(snp_genotypes)
    p = mean_val / 2
    std_val = (2 * p * (1 - p)) ** 0.5
    
    # Normalize the SNP
    normalized = (snp_genotypes - mean_val) / std_val
    
    # Replace missing values with 0
    X_YRI[i, :] = np.ma.filled(normalized, fill_value=0)

# Add contributions from only the first 100 causal SNPs
for i in range(100):
    Y_YRI += effect_size * X_YRI[i, :]  

# Compute chi² statistics using the normalized genotype matrix
chi2_stats = np.zeros(1000)
for i in range(1000):
    chi2_stats[i] = armitage_trend_test(X_YRI[i, :], Y_YRI)  

# Compute average χ² values
avg_chi2_causal = np.mean(chi2_stats[causal_labels])
avg_chi2_null = np.mean(chi2_stats[~causal_labels])
avg_chi2_all = np.mean(chi2_stats)

print("Average χ² for causal SNPs (first 100):", avg_chi2_causal)
print("Average χ² for null SNPs (remaining 900):", avg_chi2_null)
print("Average χ² for all SNPs:", avg_chi2_all)

Average χ² for causal SNPs (first 100): 2.2837323105419607
Average χ² for null SNPs (remaining 900): 0.9791111181833495
Average χ² for all SNPs: 1.1095732374192107


*comments*

According to theoretical derivations, the average χ² for casual SNPs should be close to 1 + hg2 * N/M. Here, hg2 is 1, N is 113 and M is 100. So the average ATT(casual) should be around 2.13. On the other hand, the average χ² for null SNPs should be close to 1. The overall average χ² for all SNPs should be 1 + hg2 * N/M, where M = 1000 in this case. So the average ATT(all) is should be about 1 + 1 * 113/1000 = 1.113. From our results, we see that the values align closely with the values derived from theory. The slight difference might be due to relatively small number of SNPs and the presence of masked values. 

## (4)

(a) Using the simulated data from (3), compute a 113 x 113 genetic relationship matrix from normalized genotypes and use H-E regression to estimate the value of hg2. (b) Following the variance components approach to estimating hg2, compute log likelihoods given the phenotypes of the following values of (σg2,σe2): (0.01,0.99), (0.10,0.90), (0.50,0.50), (0.90,0.10), (0.99,0.01).  Which value of (σg2,σe2) produces the highest likelihood?  (Note: it will be necessary to invert the genetic relationship matrix in order to compute the likelihoods.)

### Solution

In [12]:
n_snps, n_individuals = YRI_geno_subset.shape

# Initialize the normalized genotype matrix
X_YRI = np.zeros((n_snps, n_individuals))  # Ensure X_YRI has the correct shape

for i in range(1000):  # Assuming the first 1000 SNPs
    snp_genotypes = YRI_geno_subset[i, :]
    
    # Compute allele frequency and standard deviation
    mean_val = np.ma.mean(snp_genotypes)
    p = mean_val / 2
    std_val = (2 * p * (1 - p)) ** 0.5
    
    # Normalize the SNP
    normalized = (snp_genotypes - mean_val) / std_val
    
    # Replace missing values with 0 and store in X_YRI
    X_YRI[i, :] = np.ma.filled(normalized, fill_value=0)  

# Compute the Genetic Relationship Matrix (GRM)
A = X_YRI.T.dot(X_YRI) / 1000  

# Exclude the diagonal by creating a mask
mask = ~np.eye(A.shape[0], dtype=bool)  # Boolean mask where diagonal is False

# Compute SNP-heritability using H-E regression (excluding diagonal elements)
numerator = np.sum(A[mask] * np.outer(Y_YRI, Y_YRI)[mask])  
denom = np.sum(A[mask]**2)  

hg2 = numerator / denom

print(f"Estimated SNP-heritability (h_g^2) using H-E regression: {hg2}")

Estimated SNP-heritability (h_g^2) using H-E regression: 0.9787278885844746


In [13]:
Y_YRI_normalized = (Y_YRI-Y_YRI.mean())/np.std(Y_YRI)

def compute_log_likelihood(phenotype, A, sigma_g2, sigma_e2):
    N = len(phenotype)
    I = np.eye(N)
    V = sigma_g2 * A + sigma_e2 * I

    # Compute log determinant directly (assumes V is positive definite)
    logdet = np.log(np.linalg.det(V))
    V_inv = np.linalg.inv(V)
    quad_form = phenotype.T @ V_inv @ phenotype

    # Remove the constant term: N * log(2*pi)
    log_likelihood = -0.5 * (logdet + quad_form)
    
    return log_likelihood

# Given variance values
variance_pairs = [(0.01, 0.99), (0.10, 0.90), (0.50, 0.50), (0.90, 0.10), (0.99, 0.01)]

# Compute log-likelihoods
log_likelihoods = [compute_log_likelihood(Y_YRI_normalized, A, sg2, se2) for sg2, se2 in variance_pairs]

# Print results
for (sg2, se2), ll in zip(variance_pairs, log_likelihoods):
    print(f"σg² = {sg2}, σe² = {se2}, Log-Likelihood = {ll}")

σg² = 0.01, σe² = 0.99, Log-Likelihood = -56.43895443356515
σg² = 0.1, σe² = 0.9, Log-Likelihood = -55.92242728768172
σg² = 0.5, σe² = 0.5, Log-Likelihood = -54.21241952830645
σg² = 0.9, σe² = 0.1, Log-Likelihood = -53.067180992662486
σg² = 0.99, σe² = 0.01, Log-Likelihood = -52.034748414413244


*comments*

The estimated SNP-heritability (h_g^2) using H-E regression is close to the true value of 1.00 used to simulate the phenotypes. The highest likelihood is attained at σg² = 0.99, σe² = 0.01 because we use the genetic relationship matrix to simulate the phenotype, so as expected, nearly all of the variation in the phenotype is best explained by the genetic factors represented by the GRM.   

## (5)

Consider a set of 1,000 unlinked SNPs in the 113 YRI individuals defined by the first SNP and every 50th SNP thereafter (SNPs 0, 50, …, 49950, if the first SNP is SNP 0).  Assign the first 12 SNPs as causal SNPs and other 988 SNPs as non-causal SNPs. Simulate quantitative phenotypes for YRI individuals by assuming that each causal SNP has effect size per normalized genotype = sqrt(1/12) = 0.289.  Assume that only even SNPs (SNPs 0, 2, …, 998 of SNPs 0-999) are genotyped SNPs.
(a) Run a GWAS using the 500 genotyped SNPs, using the genome-wide significance threshold P = 0.05/500 = 0.0001. (Note: P<0.0001 corresponds to χ2>15.14.) What is hGWAS2, the heritability explained by genome-wide significant SNPs?  
(b) Use H-E regression to estimate hg2, the heritability explained by genotyped SNPs.
(c) What is h2?  Comment on the relative values of hGWAS2, hg2, h2.


### Solution

In [14]:
# Create a 2D matrix to store normalized genotypes
X_YRI_2 = np.zeros_like(YRI_geno_subset, dtype=float)

# Standardize all SNPs and compute phenotype contribution
for i in range(1000):  
    snp_genotypes = YRI_geno_subset[i, :]
    
    # Compute allele frequency and standard deviation
    mean_val = np.ma.mean(snp_genotypes)
    p = mean_val / 2
    std_val = (2 * p * (1 - p)) ** 0.5
    
    # Normalize the SNP
    normalized = (snp_genotypes - mean_val) / std_val
    
    # Replace missing values with 0
    X_YRI_2[i, :] = np.ma.filled(normalized, fill_value=0)

# Initialize an array to hold the phenotype for each individual.
effect_size_2 = np.repeat([np.sqrt(1/12), 0], [12, 988])

Y_YRI_2 = X_YRI_2.T.dot(effect_size_2)

# Even-indexed SNPs
genotyped_indices = np.arange(0, 1000, 2)

# Re-calculate chi-square statistics
chi2_stats_2 = np.array([armitage_trend_test(X_YRI_2[i, :], Y_YRI_2) for i in genotyped_indices])

# Update p-value calculation
p_values = 1 - chi2.cdf(chi2_stats_2, df=1)

# Significant SNPs by p-value
significant_by_p = np.where(p_values < 0.0001)[0]
print(f"Significant SNPs by p-value: {len(significant_by_p)}")

# Check chi-square threshold consistency
threshold = chi2.ppf(0.9999, df=1)
significant_by_chi2 = chi2_stats_2 >= threshold

# Calculate h2GWAS
h_GWAS_2 = np.sum(chi2_stats_2[significant_by_chi2]) / 113
print(f"hGWAS2 = {h_GWAS_2}")

Significant SNPs by p-value: 1
hGWAS2 = 0.1349942474912217


In [15]:
X_YRI_2_genotyped = X_YRI_2[genotyped_indices, :]

# Compute the Genetic Relationship Matrix (GRM) using only genotyped SNPs
A_2 = X_YRI_2_genotyped.T.dot(X_YRI_2_genotyped) / len(genotyped_indices)

# Exclude the diagonal by creating a mask
mask = ~np.eye(A_2.shape[0], dtype=bool)  

# Compute SNP-heritability using H-E regression (excluding diagonal elements)
numerator = np.sum(A_2[mask] * np.outer(Y_YRI_2, Y_YRI_2)[mask])  
denom = np.sum(A_2[mask]**2)  

hg2 = numerator / denom

print(f"Estimated SNP-heritability (h_g^2) for 500 genotyped SNPs using H-E regression: {hg2}")

Estimated SNP-heritability (h_g^2) for 500 genotyped SNPs using H-E regression: 0.5458249729665527


*comments*

Since half of the causal SNPs are not genotyped, the estimated SNP-heritability (h_g^2) for 500 genotyped SNPs using H-E regression is much smaller than 1. Also, some typed causal SNPs are not genome-wide significant. In fact, only 1 SNP was found to be genome-wide significant, so h_GWAS^2 (heritability explained by P<5x10-8 GWAS SNPs) is much smaller than h_g^2. Therefore, h_GWAS^2 < h_g^2 < h^2, and h^2 ~ 1.  